## Case 1 - Deterministic
## Case 2 - DRO opt w/o $\gamma$
## Case 3 - DRO opt w $\gamma$
## Case 4 - DRO opt w DRJCC
## Case 5 - DRO opt DRJCC with ESS reserve
## Case 6 - DRO opt DRJCC with ESS reserve but without revenue
### Author: Junhyeok Kim

In [1]:
# Import the library

import os
import pandas as pd
import numpy as np
import sys
import time

import matplotlib.pyplot as plt
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})

In [2]:
# Define the Variables
BASE_DIR = os.getcwd()
os.chdir("../")
path = os.getcwd() 
sys.path.append(path) # 폴더 한 단계 위에서 file import 하기 위해서 sys path 설정
sys.path.append(f"{path}/src")
sys.path.append(f"{path}/src/Data_Generation")

from rts_data import generate_wind, generate_gen_dict
from aggregator import aggregator
from gurobiMILP import gurobi_MILP
from draw_fig import Single_Case_Plot

In [3]:
### Parameters
DRO_param = {'eps_joint_cvar' : 0.05}
DRO_param['theta'] = 0.05

# Vector for Bonferroni approximation
rho_vectorC = np.linspace(0, 0.0025, 26)

# Number of individual runs (number of coupled datasets in the numerical study)

IR_max = 100
IR_sim = 100

# Number of out of sample data for each individual run (N') for testing
# dataset

OOS_max = 200
OOS_sim = 100

# Number of maximum sample size (N)

N_max = 1000

# Number of sample data in training dataset (N)

N = 100;


# Total number of data 

n_total_scen = IR_max * (N_max + OOS_max)

In [4]:
# Define the Parameters

# case_dict: Generate various scenario
# res_var: Define the WT and PV as variables [True] or parameters [False]
# case:
    # case 1 : w/o uncertainty
    # case 2 : w uncertainty with DRO
    #UNIT_TIME: 1 -> 1 hour
    
case_dict = {'case':6, 'UNIT_TIME': 1, 'bid_type':'deterministic', 'n_total_scen': n_total_scen, 'N_max': N_max, 'OOS_max':OOS_max,
             'IR_max': IR_max, 'N': N, 'OOS_sim': OOS_sim, 'divide_factor': 2}

case_dict['date'] = '20220911'

nTimeslot = int(24 / case_dict['UNIT_TIME'])
char_ess = {'initSOC':0.5, 'termSOC':0.5, 'minSOC':0.2, 'maxSOC':0.8, 'efficiency':0.95}


model_dict = {'nVPP':1, 'path': path, 'N_PIECE': 10, 'nTimeslot': nTimeslot}

# Once, set te PV, WT, SMP uncertainties identically in each time (PV: 5%, WT: 10%, SMP: 10%)
uncertainty_dict = {'pv': np.ones(nTimeslot)*0.10 , 'wt': np.ones(nTimeslot)*0.10, 'smp':np.ones(nTimeslot)*0.10}


if case_dict['case']==2:
    
    model_dict['uncertainty'] = uncertainty_dict
    case_dict['bid_type'] = 'risky'
    
elif case_dict['case'] == 1:
    
    uncertainty_dict = {'pv': np.zeros(nTimeslot), 'wt': np.zeros(nTimeslot), 'smp':np.zeros(nTimeslot)}
    model_dict['uncertainty'] = uncertainty_dict
    

In [5]:
# Set Up VPP  #Unit: kW

vpp_list = []
for i in range(model_dict['nVPP']):
    nGen_dict = {'WT':4,'PV':2, 'ESS':1, 'DG':2}
    wt_list = [800, 700, 900, 1000]
    pv_list = [500, 750, 500, 700]
    ess_list = [300]
    capacity_list = [1500]
    dg_list = [300, 400, 900]
    max_list = [wt_list, pv_list, ess_list, capacity_list, dg_list]
    
    dg_dict_list = []
    for j in range(nGen_dict['DG']):
        dg_dict_list.append(generate_gen_dict(j,dg_list[j], model_dict))
    model_dict['dg_dict_list'] = dg_dict_list
    
    agg_dict = {'name': f'cvpp{i+1}', 'code': f'xds{i+1}', 'gen':nGen_dict}
    vpp_list.append(aggregator(agg_dict, char_ess, model_dict, case_dict))
    vpp_list[i].set_der_power(max_list)
    vpp_list[i].set_smp_data(case_dict['date'])
    
vpp_list[0].get_res_table()

Error
'aggregator' object has no attribute 'wt_uncert'
Aggregator set_res_table method
Uncertainty does not exist


,name,type,number,min_power,max_power,capacity
0,WT1_cvpp1,WT,1,0.0,800,NaN
1,WT2_cvpp1,WT,2,0.0,700,NaN
2,WT3_cvpp1,WT,3,0.0,900,NaN
3,WT4_cvpp1,WT,4,0.0,1000,NaN
4,PV5_cvpp1,PV,5,0.0,500,NaN
5,PV6_cvpp1,PV,6,0.0,750,NaN
6,ESS7_cvpp1,ESS,7,-300.0,300,1500.0
7,DG8_cvpp1,DG,8,30.0,300,NaN
8,DG9_cvpp1,DG,9,40.0,400,NaN


In [6]:
# Gurobi Optimization Model
Wmax = vpp_list[0].wt_list[0].max_power 
Wmu = vpp_list[0].wt_list[0].profile_mu
#case_dict['theta'] = DRO_param['theta']* Wmu
#case_dict['theta'] = np.reshape(case_dict['theta'], -1)
case_dict['theta'] = [DRO_param['theta']] * nTimeslot
case_dict['eps'] = DRO_param['eps_joint_cvar']
case_dict['beta'] = 0.1
case_dict['alpha_max'] = 0.2
case_dict['GRID_PIECE'] = 100
#case_dict['theta'] = np.array([0.05]*24)

opt_bid = gurobi_MILP('opt bid', vpp_list[0], model_dict, case_dict)

mip_gap = 0.0001
feas_tol = 1e-4
sol, obj_dict, P_dict, U_dict, slack_dict = opt_bid.optimize(mip_gap, feas_tol)

Does not Cosidered alpha
Add Bid Constraint
start set_dro_obj_constriants
end set_dro_obj_constriants
start drjcc
iteration 0 of sum drjcc
iteration 5 of sum drjcc
iteration 10 of sum drjcc
iteration 15 of sum drjcc
iteration 20 of sum drjcc
finish max constraint of DRJCC
start set_base_objectives
end set_base_objectives
Set parameter FeasibilityTol to value 0.0001
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24547 rows, 56592 columns and 105078 nonzeros
Model fingerprint: 0x96aaf7f2
Model has 4800 general constraints
Variable types: 56400 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-02, 5e+03]
  Bounds range     [1e+00, 1e+07]
  RHS range        [7e-02, 3e+05]
Presolve removed 18016 rows and 49695 columns
Presolve tim


Cutting planes:
  Learned: 5
  Gomory: 29
  Cover: 104
  Implied bound: 696
  MIR: 288
  Flow cover: 594
  Inf proof: 19
  Zero half: 2
  Network: 4
  RLT: 12
  Relax-and-lift: 61
  BQP: 1

Explored 2953 nodes (61554 simplex iterations) in 7.32 seconds (5.11 work units)
Thread count was 16 (of 16 available processors)

Solution count 10: 4.33353e+06 4.32772e+06 4.18113e+06 ... 4.02819e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 4.333531756367e+06, best bound 4.333531756367e+06, gap 0.0000%
Optimal Solution:
Optimization Duration Time: 7.347994327545166


In [7]:
opt_bid.is_igdt_risk_averse

False

In [8]:
obj1 = obj_dict['obj1']
print(obj1)

try:
    obj2 = obj_dict['obj2']
    obj3 = obj_dict['obj3']
    print(obj2)
    print(obj3)
    
except:
    print("no obj2, obj3")

try:
    obj3_full = obj_dict['obj3_full']
except:
    pass
print(sum(obj1))
print(sum(obj2))
print(sum(obj3))

print(sum(sum(obj_dict['dg_cost'])))

print(opt_bid.m.objVal)

[ 188327.408905  186891.371695  131185.338461  131803.382557
  128836.753134  133330.823620  195890.673889  207703.513837
  182307.694523  244604.808921  192666.585201  201978.252511
  256212.114761  258229.492231  229028.705266  250984.556951
  421095.000232  415234.841116  393021.762963  390800.344397
  391144.576186  389665.680674  389255.525386  249631.090776]
[ 7190.500000  7175.500000  6605.000000  6586.500000  6589.000000
  6629.000000  7673.000000  7673.000000  7195.500000  7127.500000
  6476.500000  6476.500000  7167.000000  7194.500000  7200.000000
  7430.000000  11297.000000  12245.500000  12246.000000  12249.000000
  12246.000000  12246.000000  12246.000000  8332.000000]
[ 0.000000  0.000000 -0.000000  0.000000 -0.000000  0.000000 -0.000000
  0.000000 -0.000000 -0.000000 -0.000000 -0.000000 -0.000000 -0.000000
  0.000000  0.000000  0.000000  0.000000 -0.000000  0.000000 -0.000000
  0.000000 -0.000000 -0.000000]
6159830.298191894
205496.5
1.837197061149709e-12
1620802.041825

In [9]:
print(P_dict['sum_dg'])

if opt_bid.is_dg_reserve:
    print(P_dict['dg_ru'])

[[ 210.000000  210.000000  210.000000  210.000000  210.000000  210.000000
   210.000000  210.000000  210.000000  210.000000  210.000000  210.000000
   210.000000  210.000000  210.000000  210.000000  210.000000  210.000000
   210.000000  210.000000  210.000000  210.000000  210.000000  210.000000]
 [ 240.000000  240.000000  240.000000  240.000000  240.000000  240.000000
   240.000000  240.000000  242.233586  240.000000  240.000000  240.407125
   240.000000  240.000000  240.000000  240.000000  240.000000  245.523523
   251.047045  258.313064  256.876037  261.706661  263.046381  240.000000]]
[[ 90.000000  90.000000  90.000000  90.000000  90.000000  90.000000
   90.000000  65.928205  90.000000  90.000000  80.409955  90.000000
   90.000000  90.000000  90.000000  90.000000  85.381982  90.000000
   90.000000  90.000000  90.000000  90.000000  90.000000  90.000000]
 [ 160.000000  160.000000  160.000000  160.000000  160.000000  160.000000
   160.000000  157.766414  157.766414  160.000000  159.592

In [10]:
print(P_dict['essDis'])
print(P_dict['essChg'])
if opt_bid.is_ess_reserve:
    print(P_dict['RU_essdis'])
    print(P_dict['RD_esschg'])

[[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  300.000000  297.977128  300.000000  300.000000
   300.000000  300.000000  300.000000  203.920983]]
[[ 0.000000  0.000000  300.000000  300.000000  300.000000  266.443500
   0.000000  0.627451  300.000000  0.000000  300.000000  300.000000
   76.608454  28.360108  196.916012  7.481095  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000]]
[[ 26.572847  23.665258  0.000000  0.000000  3.192414  0.000000  1.013382
   0.000000  0.000000  0.119166  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  2.022872  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]]
[[ 300.000000  300.000000  0.000000  0.000000  0.000000  33.556500
   300.000000  299.372549  0.000000  300.000000  0.000000  0.000000
   223.391546  271.639892  103.083988  292.518905  300.000000  300.000000
   30

In [11]:
if opt_bid.is_bid_DRCC:
    lhs_array, rhs_array, check_array, ratio = opt_bid.check_drcc_constraint()
    print("Ratio is :", ratio)
    for i in range(24):
        print(sum(check_array[i,:]))

Ratio is : 0.97875
100.0
100.0
100.0
100.0
100.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
96.0
97.0
100.0
100.0
100.0
100.0
100.0
100.0


In [ ]:
from draw_fig import Single_Case_Plot

In [ ]:
case_fig = Single_Case_Plot(vpp_list, opt_bid, model_dict, case_dict, path)

In [ ]:
case_fig.make_plot(P_dict, slack_dict, save_flag=True)

In [ ]:
worst_bid = np.zeros(24)

for i in range(24):
    worst_bid[i] = slack_dict['lambda_obj'][i]/case_fig.da_smp[i]
print(worst_bid)

In [ ]:
if opt_bid.is_ess_reserve or opt_bid.is_dg_reserve:
    case_fig.make_reserve_plot(P_dict, save_flag=True)

In [ ]:
case_dict['case'] = 1


opt_bid_base = gurobi_MILP('opt bid_base', vpp_list[0], model_dict, case_dict)

mip_gap = 0.0001
feas_tol = 1e-4
sol_base, obj_dict_base, P_dict_base, U_dict_base, slack_dict_base = opt_bid_base.optimize(mip_gap, feas_tol)

In [ ]:

all_obj_dicts = [[None] * 10 for _ in range(10)]
all_P_dicts = [[None] * 10 for _ in range(10)]
all_U_dicts = [[None] * 10 for _ in range(10)]
all_objVal = [[None] * 10 for _ in range(10)]
all_solve_time = [[None] * 10 for _ in range(10)]
all_reliable = [[None] * 10 for _ in range(10)]
all_reliable_array = [[None] * 10 for _ in range(10)]

case_dict['case'] = 6
start_time = time.time()
for i in range(10):
    for j in range(10):
        print("*"*30)
        print("iterion i,j :", i,j)
        print("iterion i,j :", i,j)
        print("iterion i,j :", i,j)
        print("*"*30)
        DRO_param['eps_joint_cvar'] = i*0.01 + 0.01
        DRO_param['theta'] = j*0.01 + 0.01
        
        
        # Gurobi Optimization Model
        case_dict['theta'] = [DRO_param['theta']] * nTimeslot
        case_dict['eps'] = DRO_param['eps_joint_cvar']
        #case_dict['theta'] = np.array([0.05]*24)

        opt_bid = gurobi_MILP(f'opt bid{i}_{j}', vpp_list[0], model_dict, case_dict)
        sol, obj_dict, P_dict, U_dict, slack_dict = opt_bid.optimize(mip_gap, feas_tol)
        
        all_obj_dicts[i][j] = obj_dict
        all_P_dicts[i][j] = P_dict
        all_U_dicts[i][j] = U_dict
        all_objVal[i][j] = opt_bid.m.objVal
        all_solve_time[i][j] = opt_bid.opt_solve_time
        if opt_bid.is_bid_DRCC:
            lhs_array, rhs_array, check_array, ratio = opt_bid.check_drcc_constraint()
            all_reliable_array[i][j] = check_array
            all_reliable[i][j] = ratio * 100
        else:
            print("No is_bid_DRCC")
            
end_time = time.time()
total_solution_time = end_time - start_time

In [ ]:
total_solution_time

In [ ]:
df = pd.DataFrame(all_reliable)
df1 = pd.DataFrame(all_solve_time)
df2 = pd.DataFrame(all_objVal)

# Specify the Excel file path where you want to export the data
excel_file = f'{path}/result/output_reliable.xlsx'
excel_file1 = f'{path}/result/output_solve_time.xlsx'
excel_file2 = f'{path}/result/output_objVal.xlsx'
# Use the to_excel method to export the DataFrame to an Excel file
df.to_excel(excel_file, index=False)
df.to_excel(excel_file1, index=False)
df.to_excel(excel_file2, index=False)

In [ ]:
df1 = pd.DataFrame(all_reliable,all_solve_time)

In [ ]:
df1

In [ ]:
# Since the actual data is not provided, we will create a sample dataframe with random numbers
# to mimic the structure based on the image provided.
x_label = r'$\theta$'
y_label = r'$\epsilon$'
z_label = 'Solution Time [s]'
title = r"Solution Time  via ($\theta$, $\epsilon$)"
file_name = 'Solution Time'
label_name = [x_label, y_label, z_label, title, file_name]

# Create a sample dataframe
data = all_solve_time  # 10x10 matrix as seen in the image
# case_fig.path = "C:/Users/user/OneDrive/1. CODE/IGDT-DRO-optimal-bidding-of-virtual-power-plant"
case_fig.make_3d_plot(data, label_name, save_flag = True)

In [ ]:
x_label = r'Radius: $\theta$'
y_label = r'Viloation Rate: $\epsilon$'
z_label = 'Solution Time [s]'
title = r"Solution Time  via $\theta$, $\epsilon$"
file_name = 'Soltuion Time'
label_name = [x_label, y_label, z_label, title, file_name]

# Create a sample dataframe
data = all_solve_time  # 10x10 matrix as seen in the image

x_label = label_name[0]
y_label = label_name[1]
z_label = label_name[2]
title = label_name[3]
file_name = label_name[4]

df = pd.DataFrame(data)

# Initialize a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Create coordinate arrays for the dataframe values
X = np.arange(df.shape[1])*0.01 + 0.01
Y = np.arange(df.shape[0])*0.01 + 0.01
X, Y = np.meshgrid(X, Y)
Z = df.values

# Plotting the 3D surface plot
surf = ax.plot_surface(X, Y, Z, cmap='viridis')

# Add color bar which maps values to colors
fig.colorbar(surf, shrink=0.5, aspect=5, pad=0.1)

# Set labels
ax.set_xlabel(x_label, labelpad=10) # labelpad 값을 조정하여 레이블과 눈금 사이의 거리를 조절할 수 있습니다.
ax.set_ylabel(y_label, labelpad=10) # labelpad 값을 조정하여 레이블과 눈금 사이의 거리를 조절할 수 있습니다.
ax.set_zlabel(z_label, labelpad=5)

# Show the plot
plt.savefig("C:/Users/user/OneDrive/1. CODE\IGDT-DRO-optimal-bidding-of-virtual-power-plant/fig/test.png", dpi = 500, bbox_inches='tight')
plt.show()


In [ ]:
# Since the actual data is not provided, we will create a sample dataframe with random numbers
# to mimic the structure based on the image provided.

# Create a sample dataframe
data = all_objVal  # 10x10 matrix as seen in the image

x_label = r'$\theta$'
y_label = r'$\epsilon$'
z_label = 'Obj'
title = r"Objective Value via $\theta$, $\epsilon$"
file_name = 'wdrcc_objective_value'
label_name = [x_label, y_label, z_label, title, file_name]

# Create a sample dataframe
case_fig.make_3d_plot(data, label_name, save_flag = True)

In [ ]:
# Since the actual data is not provided, we will create a sample dataframe with random numbers
# to mimic the structure based on the image provided.

# Create a sample dataframe
data = all_reliable  # 10x10 matrix as seen in the image
data = np.array(data).transpose()

x_label = r'$ \epsilon$'
y_label = r'$\theta$'
z_label = 'Reliability [%]'
title = r"Reliability $(1-\rho) \times 100$ [%] via $\theta$, $\epsillon$"
file_name = 'reliability'
label_name = [x_label, y_label, z_label, title, file_name]

# Create a sample dataframe
case_fig.make_3d_plot(data, label_name, save_flag = True)

In [ ]:
# Since the actual data is not provided, we will create a sample dataframe with random numbers
# to mimic the structure based on the image provided.

# Create a sample dataframe
data = all_reliable  # 10x10 matrix as seen in the image
data = np.array(data).transpose()
df = pd.DataFrame(data)

# Initialize a 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Create coordinate arrays for the dataframe values
X = np.arange(df.shape[1])*0.01 + 0.01
Y = np.arange(df.shape[0])*0.01 + 0.01
X, Y = np.meshgrid(X, Y)
Z = df.values

# Plotting the 3D surface plot
surf = ax.plot_surface(X, Y, Z, cmap='viridis')

# Add color bar which maps values to colors
fig.colorbar(surf, shrink=0.5, aspect=5)

# Set labels
ax.set_ylabel(r'$\theta$', fontsize=12)
ax.set_xlabel(r'$ \epsilon$', fontsize=12)
ax.set_zlabel(r'$\rho $ [%]', fontsize=12)
ax.set_title("Reliability via theta, epsillon", fontsize=16)

# Show the plot
plt.show()

In [ ]:
# To make the surface plot smoother, we can interpolate the data
# to create a finer grid before plotting.

from scipy.interpolate import griddata

# Create a dataframe again
data = all_reliable  # 10x10 matrix as seen in the image
data = np.array(data).transpose()
df = pd.DataFrame(data)

# Create the grid coordinates for interpolation
X = np.arange(df.shape[1])
Y = np.arange(df.shape[0])
X, Y = np.meshgrid(X, Y)
Z = df.values.flatten()

# Interpolation grid
xi = np.linspace(X.min(), X.max(), 500)
yi = np.linspace(Y.min(), Y.max(), 500)
xi, yi = np.meshgrid(xi, yi)

# Interpolate; this creates a smooth surface
Zi = griddata((X.flatten(), Y.flatten()), Z, (xi, yi), method='cubic')

# Plotting the smooth surface
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(xi, yi, Zi, cmap='viridis', edgecolor='none')

# Color bar
fig.colorbar(surf, shrink=0.5, aspect=5)

# Labels
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# Show the plot
plt.show()

In [ ]:
total_solution_time